In [1]:
# import libraries 
import pandas as pd
import matplotlib.pyplot as plt
import os 
import math
import sys
from pathlib import Path
import seaborn as sns

# Add 'src' to the system path
sys.path.append(str(Path().resolve() / 'src'))

# Load dataframes

In [2]:
df_movies = pd.read_csv('data/processed/movies.csv')
df_movies.head()

,Wikipedia movie ID,Freebase ID,Movie name,Release date,Box office revenue,Runtime,Languages,Countries,Genres
0,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
1,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
2,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
3,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}"
4,13696889,/m/03cfc81,The Gangsters,1913-05-29,NaN,35.0,"{""/m/06ppq"": ""Silent film"", ""/m/02h40lc"": ""Eng...","{""/m/09c7w0"": ""United States of America""}","{""/m/02hmvc"": ""Short Film"", ""/m/06ppq"": ""Silen..."


In [3]:
df_characters = pd.read_csv('data/processed/characters.csv')
df_characters.head()

,Wikipedia movie ID,Freebase movie ID,Movie release date,Character name,Actor DoB,Actor gender,Actor height (m),Actor ethnicity (Freebase ID),Actor name,Actor age at movie release,Freebase character/actor map ID,Freebase character ID,Freebase actor ID
0,975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4
1,975900,/m/03vyhn,2001-08-24,Desolation Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l
2,975900,/m/03vyhn,2001-08-24,Sgt Jericho Butler,1967-09-12,M,1.750,NaN,Jason Statham,33.0,/m/02vchl6,/m/0bgchnq,/m/034hyc
3,975900,/m/03vyhn,2001-08-24,Bashira Kincaid,1977-09-25,F,1.650,NaN,Clea DuVall,23.0,/m/02vbb3r,/m/0bgchp9,/m/01y9xg
4,975900,/m/03vyhn,2001-08-24,Commander Helena Braddock,1949-05-26,F,1.727,/m/0x67,Pam Grier,52.0,/m/02vdcfp,/m/0bgchnd,/m/0418ft


In [4]:
df_plots = pd.read_csv('data/processed/plot_summaries.csv')
df_plots.head()

,Wikipedia movie ID,Summary
0,31186339,The nation of Panem consists of a wealthy Capi...
1,20663735,Poovalli Induchoodan is sentenced for six yea...
2,2231378,"The Lemon Drop Kid , a New York City swindler,..."
3,595909,Seventh-day Adventist Church pastor Michael Ch...
4,5272176,The president is on his way to give a speech. ...


In [5]:
# merge movie metadata and plots 
df_movies_summaries = pd.merge(df_movies, df_plots, on='Wikipedia movie ID')
df_movies_summaries.head()

,Wikipedia movie ID,Freebase ID,Movie name,Release date,Box office revenue,Runtime,Languages,Countries,Genres,Summary
0,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic...",A series of murders of rich young women throug...
1,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}","Eva, an upper class housewife, becomes frustra..."
2,18998739,/m/04jcqvw,The Sorcerer's Apprentice,2002,NaN,86.0,"{""/m/02h40lc"": ""English Language""}","{""/m/0hzlz"": ""South Africa""}","{""/m/0hqxf"": ""Family Film"", ""/m/01hmnh"": ""Fant...","Every hundred years, the evil Morgana returns..."
3,6631279,/m/0gffwj,Little city,1997-04-04,NaN,93.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06cvj"": ""Romantic comedy"", ""/m/0hj3n0w"": ...","Adam, a San Francisco-based artist who works a..."
4,171005,/m/016ywb,Henry V,1989-11-08,10161099.0,137.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/04xvh5"": ""Costume drama"", ""/m/082gq"": ""Wa...",{{Plot|dateAct 1Act 2Act 3Act 4Act 5 Finally n...


# Preprocessing - Data Distributions 

In [6]:
df_tmdb = pd.read_csv('data/processed/TMDB_movie_dataset_v11.csv')
df_tmdb = df_tmdb[['title', 'status','release_date',
       'revenue', 'runtime', 'budget', 'overview','genres',
       'production_companies', 'production_countries', 'keywords']]

In [7]:
df_tmdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128267 entries, 0 to 1128266
Data columns (total 11 columns):
 #   Column                Non-Null Count    Dtype 
---  ------                --------------    ----- 
 0   title                 1128254 non-null  object
 1   status                1128267 non-null  object
 2   release_date          944893 non-null   object
 3   revenue               1128267 non-null  int64 
 4   runtime               1128267 non-null  int64 
 5   budget                1128267 non-null  int64 
 6   overview              900651 non-null   object
 7   genres                677445 non-null   object
 8   production_companies  509961 non-null   object
 9   production_countries  629797 non-null   object
 10  keywords              306760 non-null   object
dtypes: int64(3), object(8)
memory usage: 94.7+ MB


In [8]:
df_tmdb['genres'].explode().tolist()


['Action, Science Fiction, Adventure',
 'Adventure, Drama, Science Fiction',
 'Drama, Action, Crime, Thriller',
 'Action, Adventure, Fantasy, Science Fiction',
 'Science Fiction, Action, Adventure',
 'Action, Adventure, Comedy',
 'Adventure, Action, Science Fiction',
 'Drama',
 'Action, Science Fiction, Adventure',
 'Thriller, Crime',
 'Comedy, Drama, Romance',
 'Adventure, Fantasy',
 'Action, Science Fiction, Adventure',
 'Drama, Western',
 'Drama, Crime',
 'Adventure, Science Fiction, Action',
 'Action, Science Fiction',
 'Drama, Romance',
 'Crime, Thriller, Drama',
 'Adventure, Fantasy, Action',
 'Adventure, Fantasy, Action',
 'Drama, Thriller, Mystery',
 'Crime, Drama, Comedy',
 'Action, Adventure, Science Fiction',
 'Adventure, Action, Science Fiction',
 'Action, Crime, Drama, Thriller',
 'Action, Adventure, Science Fiction',
 'Action, Adventure, Science Fiction',
 'Action, Adventure, Fantasy',
 'Action, Adventure, Science Fiction',
 'Drama, Thriller, War',
 'Science Fiction, Adve

In [9]:
df_tmdb.shape

(1128267, 11)

In [10]:
test.shape

NameError: name 'test' is not defined

In [26]:
df_tmdb['status'].value_counts()

status
Released           1102948
In Production        10047
Post Production       7942
Planned               6683
Rumored                344
Canceled               303
Name: count, dtype: int64

In [51]:
# Keep only the released movies
df_tmdb = df_tmdb[df_tmdb['status'] == 'Released']

In [52]:
df_tmdb['revenue'].shape

(1102948,)

In [54]:
test = 
test['revenue'].shape

(1082327,)

In [55]:
test = df_tmdb[df_tmdb['revenue']>=0]
test['revenue'].shape


(1102947,)

In [29]:
test = df_tmdb[df_tmdb['release_date'].str.len()!=10]
test['release_date'].shape


(166521,)

In [32]:
df_tmdb['release_date'] = pd.to_datetime(df_tmdb['release_date'])
df_tmdb['release_date'].dropna(inplace=True)
df_tmdb.info()


<class 'pandas.core.frame.DataFrame'>
Index: 1102948 entries, 0 to 1128266
Data columns (total 11 columns):
 #   Column                Non-Null Count    Dtype         
---  ------                --------------    -----         
 0   title                 1102937 non-null  object        
 1   status                1102948 non-null  object        
 2   release_date          936427 non-null   datetime64[ns]
 3   revenue               1102948 non-null  int64         
 4   runtime               1102948 non-null  int64         
 5   budget                1102948 non-null  int64         
 6   overview              881019 non-null   object        
 7   genres                658951 non-null   object        
 8   production_companies  495932 non-null   object        
 9   production_countries  614086 non-null   object        
 10  keywords              300428 non-null   object        
dtypes: datetime64[ns](1), int64(3), object(7)
memory usage: 101.0+ MB


In [35]:
from src.data.process_data import *

## Movies 
### Languages

In [38]:
df_movies["Lang names"] = df_movies["Languages"].apply(extract_tuples_values)
df_movies["Lang names"].head()

0                 [English Language]
1               [Norwegian Language]
2                 [English Language]
3                  [German Language]
4    [Silent film, English Language]
Name: Lang names, dtype: object

### Genres
### Release dates
### Runtimes 
### Box office revenues
### Countries 
## Actors 
### Gender
### Ages (DoB and age during movie) 
### Ethnicity 
### Height?

# Preprocessing - Cleaning 